## 1. Pipeline

In [ ]:
from transformers import pipeline

model_id = ""
model_pipeline = pipeline("text-classification", model=model_id)
model_pipeline(dataset["title"][:5])

## 2. 직접 추론하기

In [ ]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class CustomPipeline:
    def __init__(self, model_id):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model.eval()

    def __call__(self, texts):
        tokenized = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**tokenized)
            logits = outputs.logits
        
        probs = softmax(logits, dim=1)
        scores, labels = torch.max(probs, dim=1)
        labels_str = [self.model.config.id2label[label_idx] for label_idx in labels.tolist()]
        return [{"label": label, "score": score.item()} for label, score in zip(labels_str, scores)]
    
custom_pipeline = CustomPipeline(model_id)
custom_pipeline(dataset["title"][:5])